In [ ]:
"""
Prepare calving netcdf based on Rignot
"""

In [ ]:
import xarray as xr
import pandas as pd

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachine_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
inputpath_csv = home_path+'DATA/SUMMER_PAPER/raw/'
Rignot_data = pd.read_csv(inputpath_csv + 'Rignot_data.csv',delimiter=';',header=[0], index_col=[0])

In [ ]:
Rignot_data

In [ ]:
calvflux_list = []
for kisf in file_isf.Nisf:
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    if nname in Rignot_data.index.values:
        calvflux_list.append(float(Rignot_data['Ice-front flux'][nname]))
    elif nname == 'Ross':
        calvflux_list.append(float(Rignot_data['Ice-front flux']['Ross East']) + float(Rignot_data['Ice-front flux']['Ross West']))
    elif nname == 'Filchner-Ronne':
        calvflux_list.append(float(Rignot_data['Ice-front flux']['Filchner']) + float(Rignot_data['Ice-front flux']['Ronne']))
    else:
        print(nname)
calvflux_array = xr.DataArray(data=np.array(calvflux_list), dims=['Nisf']).assign_coords({'Nisf': file_isf.Nisf})

In [ ]:
calvflux_array.to_dataset(name='calving_flux_rignot').to_netcdf(inputpath_csv + 'calving_flux_rignot13.nc')